## Class Activation Maps -- CAM

* This method applied only on CNN models, to make the decesion more tranperent 
* It is a local explainer 
* No applied to tupular data, only images 

## How it works

* You take the replresentations in the last CONV layer
* Also, take the prediction from the clasification layer, the dense one 
  this is a probalility vector with the same leght of the classes 
* This two items will form the CAM model, cam_model
* Build a class activation map, which is a matrix that shows which part of the image made the model decides for a certain class 


* the Cam model to predict, it will return the to lists, the features and the predictions 
* if you would like to build the activation map for an image, you takes its features using its index and the its prediction by applying argmax function to the to its probability vector extratced from the dense layer 
* Zoom, or resize the feature to the original size of the image 
* Take the learned weights of the global average activation layer
* Dot product of the the resized featuers and the weights/activations of the model, 
* The results of this dot procuct is the, what we call class activation map



# The issue with method 

* Local
* Work for image modality 

# Cat vs. Dog clasification

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import keras
from keras.models import Sequential, Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cv2

In [ ]:
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

In [ ]:
train_data = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised=True)
validation_data = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised=True)
test_data = tfds.load('cats_vs_dogs', split='train[-10%:]', as_supervised=True)

In [ ]:
def normalize(image, label):
  
  # cast to float
  image = tf.cast(image, tf.float32)
  # normalize the pixel values
  image = (image/255)
  # resize to 300 x 300
  image = tf.image.resize(image,(300,300))

  return image, label

# use the utility function above to preprocess the images
train_batches = train_data.map(normalize).batch(32)

In [ ]:
model = Sequential()
model.add(Conv2D(16, input_shape=(300, 300, 3), kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,  2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))

## add A golbalAveragePooling
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
train_batches = tf.data.experimental.save(train_batches, '/content/drive/MyDrive/XAI workshop/datasets/train_data_prepared_cvd')

In [ ]:
model.summary()

In [ ]:
# Training will take around 30 minutes to complete using a GPU. Time for a break!

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001))
model.fit(train_batches, epochs=25)

In [ ]:
model.save('/content/drive/MyDrive/XAI workshop/Models/cnn_model_cam.h5')